In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from task1_data_preprocessing import preprocess_data
%matplotlib inline

prices = preprocess_data()["pivot_close"][['TSLA','BND','SPY']].sort_index()
start = "2024-08-01"; end = "2025-07-31"
prices_bt = prices.loc[start:end]
prices_bt.head()


In [ ]:
def run_backtest_weights(prices, weights, start, end, initial_capital=100000, mode='hold'):
    prices = prices.loc[start:end].dropna(how='all')
    dates = prices.index
    # start allocation at first day
    first = dates[0]
    # target dollar allocation
    port_val = initial_capital
    target_dollars = {a: weights.get(a,0)*port_val for a in ["TSLA","BND","SPY"]}
    holdings = {a: target_dollars[a]/prices.loc[first,a] if prices.loc[first,a]>0 else 0 for a in target_dollars}
    nav = []
    for d in dates:
        nav_val = sum(holdings[a]*prices.loc[d,a] for a in holdings)
        nav.append((d, nav_val))
        # handle monthly rebalance if desired (left as exercise in notebook)
    nav_df = pd.DataFrame(nav, columns=['date','nav']).set_index('date')
    nav_df['returns'] = nav_df['nav'].pct_change().fillna(0)
    return nav_df

# weights example
strategy_weights = {"TSLA":0.3,"SPY":0.4,"BND":0.3}
benchmark_weights = {"TSLA":0.0,"SPY":0.6,"BND":0.4}


In [ ]:
strat_nav = run_backtest_weights(prices_bt, strategy_weights, start, end)
bench_nav = run_backtest_weights(prices_bt, benchmark_weights, start, end)

plt.figure(figsize=(12,5))
plt.plot(strat_nav.index, strat_nav['nav']/strat_nav['nav'].iloc[0], label='Strategy')
plt.plot(bench_nav.index, bench_nav['nav']/bench_nav['nav'].iloc[0], label='Benchmark 60/40', linestyle='--')
plt.legend(); plt.title('Backtest cumulative returns'); plt.show()


In [ ]:
def metrics(nav_df):
    total = nav_df['nav'].iloc[-1]/nav_df['nav'].iloc[0]-1
    days = (nav_df.index[-1]-nav_df.index[0]).days
    ann = (1+total)**(252/days)-1
    vol = nav_df['returns'].std()*np.sqrt(252)
    sharpe = (nav_df['returns'].mean()*252 - 0.03) / vol
    return {'total':total, 'annualized':ann, 'vol':vol, 'sharpe':sharpe}

print('Strategy:', metrics(strat_nav))
print('Benchmark:', metrics(bench_nav))
